In [1]:
library(randomForest)

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.


In [4]:
load('data/hw6ds1.rdata')



In [2]:
filter_chisq = function(dstrain, ypos="pos", min_count=5,
chi_threshold = 1e-5) {
 nugrams = ncol(dstrain) #number of unigram+1
 chiall = rep(-1, nugrams) 
 yvec = as.numeric(dstrain[,1]==ypos)
 options(warn = -1) #silence the warning
 for(ii in 2:nugrams) {
 tmp1=cbind(yvec, as.numeric(dstrain[,ii]>0))
 tmp1a=table(tmp1[,1], tmp1[,2])

 if(nrow(tmp1a)<2 | ncol(tmp1a)<2) {
 #stop("tmp1a table dimension too small!")
 chiall[ii] = 0
 } else if(sum(tmp1[,2])<=min_count) {
 chiall[ii] = 0
 #cat("feature", ii, "count too low, skip\n")
 } else {
 tmp2=chisq.test(tmp1a, correct=FALSE)
 chiall[ii] = tmp2$statistic
 }
 }
 options(warn = 0) #turn the warnings back on
 o1 = order(chiall, decreasing=TRUE)

 tmpind1 = chiall[o1] > chi_threshold
 if(sum(tmpind1) ==0) {

 return(list(colpos = NULL, colname=NULL, chistat=NULL))
 } else {
 o2=o1[tmpind1]
 retname = names(dstrain)[o2]
 return(list(colpos = o2, colname=retname, chistat=chiall[o2]))
 }
}

In [78]:
rf_carton <- function(dsall, folds, testfold, ypos = "pos",  chi_threshold = 0.1, grid_length = 20,  grid_type = "loglinear", rfntree = 500 ,debuglevel=0){
    
    mytest <-function (confusion){
        precision = (confusion[2,2])/(confusion[1,2] + confusion[2,2])
        recall = (confusion[2,2])/(confusion[2,1] + confusion[2,2])
        f1 = 2 * (precision * recall ) / (precision + recall)
        return(list(precision=precision, recall=recall, f1 = f1))
    }

    ycol=1
    #this is the training dataset
    dstrain_all = dsall[-folds[[testfold]],]
    dstrain_all_y = as.factor(dstrain_all[,ycol] == ypos)
    #testing dataset
    dstest = dsall[folds[[testfold]],]
    dstest_y = as.factor(dstest[,ycol] == ypos)
    if(testfold==1) {
     tunefold=10
    } else {1
     tunefold=testfold-1
    }
    #tuning dataset
    dstune = dsall[folds[[tunefold]],]
    #this is the sub-train dataset
    dstrain = dsall[-c(folds[[tunefold]], folds[[testfold]]),]
    dstrain_y = as.factor(dstrain[,ycol] == ypos)
    dstune_y = as.factor(dstune[,ycol] == ypos)

    features = filter_chisq(dstrain, ypos, chi_threshold = chi_threshold)
    dstrain_new = dstrain[features$colpos]
    dstune_new = dstune[features$colpos]
    ncol = dim(dstrain_new)[2]

    m_min = 2
    m_max = ncol
    grids = NULL
    if(grid_type == "loglinear"){
        grids = unique(round(exp(seq(log(m_min), log(m_max),length=grid_length))))
    }
    else if(grid_type == 'equal'){
        grids = unique(round(seq(m_min, m_max, length=grid_length )))
    }
    
    f1_all = c()
    best_mtry = 2
    best_f1 = -1
    best_test_result = NULL
    for(mtry in grids){
        result = randomForest(x = dstrain_new, y = dstrain_y, xtest=dstune_new,ytest= dstune_y, ntree = rfntree ,mtry = mtry)
        test_result = mytest(result$test$confusion) 
        if(test_result$f1  > best_f1){
            best_f1 = test_result$f1
            best_mtry = mtry
            best_test_result = test_result
        }
        f1_all = c(f1_all, test_result$f1)
        return(result)
    }
    new_features = filter_chisq(dstrain_all, ypos, chi_threshold = chi_threshold)
    dstrain_all = dstrain_all[new_features$colpos]
    dstest = dstest[new_features$colpos]
    final_rf_result = randomForest(x = dstrain_all, y = dstrain_all_y, xtest=dstest,ytest= dstest_y, mtry = best_mtry)
    result = list(mgrids= grids, f1_all=f1_all, best_m=best_mtry, test= mytest(final_rf_result$test$confusion), fselect=new_features)
    return(result)
}

set.seed(5556)
rftest=rf_carton(ds1, cvfold, testfold=7,debuglevel=0)


    




In [88]:
z = rftest$test$confusion
rftest$test$confusion
z[1]
z[2]
z[3]
z[4]
z[5]
z[6]

,FALSE,TRUE,class.error
FALSE,60,0,0.0000000
TRUE,14,3,0.8235294


[1] 60

[1] 14

[1] 0

[1] 3

[1] 0

[1] 0.8235294

In [68]:
length(rftest$fselect$colpos)

[1] 346

In [31]:
dim(dstrain_all)
dim(dstrain_y_all)
dim(dstest)

[1] 696 347

NULL

[1]   78 4129

In [ ]:
fin_result = randomForest(x = dstrain_all, y = dstrain_y, xtest=dstune_new,ytest= dstune_y, mtry = mtry)

In [114]:
new_features = filter_chisq(dsall, chi_threshold = chi_threshold)
new_dall = dsall[new_features$colpos]
# new_dall
new_result = randomForest(x = new_dall, y = dsall[,1],  mtry = best_mtry)

In [117]:
mytest(new_result$confusion)
new_features

$precision
[1] 0.9351852

$recall
[1] 0.7829457

$f1
[1] 0.8523207

$colpos
  [1] 4111 4061 3150 4032 2985 3510 3783 3806 2999 3309 3416 2953 3422 3937 3891
 [16] 1497 3503 4088 3920 3812 3718 3636 3644 3657 4073 3151 3387 3499 4074 3846
 [31] 3559 3498 3715 2437 3580 3674 3837 3906 4075 4067 3970 4063 4058 4079 2729
 [46] 3230 4025 3601 3924 4121 4100 3609 3787 3879 4085 4092 3904 4083 4107 4093
 [61] 4127 4089 4118 4080 4103 3690 3942 3995 3973 3241 3836 4108 4102 4105 3915
 [76] 4104 4109 4054 4123 4076 3872 3589 3625 4064 3497 3639 3691 4114 4124 4022
 [91] 3896 3908 3910 3934 4008 3996 3751 3869 4009 4030 4062 4071 4097 3913 3998
[106] 4113 4096 3382 3515 3584 3610 3635 3658 3732 3860 4059 4086 4115 3905 4003
[121] 4042 3678 3911 4004 4122 4002 4017 4078 4029 4128 3994 3999 4069 4014 4065
[136] 3887 3888 3897 3950 3953 4039 4027 4034 4106 3598 3771 3815 3936 3839 3759
[151] 3830 3912 3932 3969 4070 3850 3864 3868 3874 3893 3926 3938 3968 3976 3992
[166] 4024 4026 4049 3993 4038 4033 3729 3919 4005 3797 3854 3871 3875 3881 3946
[181] 3952 3984 4077 3838 4048 4068 4000 3611 3667 3778 3801 3802 3809 3843 3902
[196] 3930 3939 3951 3963 3985 4087 3979 4035 4044 3501 3668 3686 3741 3769 3798
[211] 3804 3825 3866 3867 3921 3978 4082 4098 3359 3554 3616 3694 3702 3727 3762
[226] 3765 3786 3788 3796 3803 3817 3821 3823 3833 3848 3884 3965 3981 3989 4056
[241] 4095 4043 4066 3194 3290 3404 3438 3583 3590 3615 3619 3621 3624 3640 3660
[256] 3675 3684 3697 3700 3711 3723 3734 3736 3739 3743 3746 3753 3756 3767 3770
[271] 3775 3790 3794 3811 3826 3835 3845 3851 3853 3859 3878 3885 3889 3890 3917
[286] 3925 4090 3962 3982 4007 4016 4019 4023 4045 4119 4053 4046 4041 4051 3907
[301] 3980 3997 3819 4047 4052 3949 3433 3513 3634 3645 3933 4001 3974 4037 4081
[316] 3677 3773 3865 3900 3901 3931 3948 3956 3983 4129 4040 4057 4091 4099 4120
[331] 3957 3958 4050 3988 3742 3754 3766 3832 3840 3922 3923 3940 3745 3944 4117
[346] 4094 3654 3971 3972 4116 3966 3990 4110 3929 4031 3935 3961 3642 3688 3733
[361] 3814 3828 3844 3847 3849 3873 3928 3945 3947 3960 3987 4012 4013 4060 4084
[376] 4028 4101 3898 3964 4125 3487 3726 3813 3820 3827 3831 3842 3855 3967 3975
[391] 4006 4020

$colname
  [1] "w4110_product"       "w4060_new"           "w3149_launch"       
  [4] "w4031_development"   "w2984_introduce"     "w3509_expand"       
  [7] "w3782_effort"        "w3805_growth"        "w2998_introduction" 
 [10] "w3308_enhance"       "w3415_open"          "w2952_family"       
 [13] "w3421_expansion"     "w3936_technology"    "w3890_research"     
 [16] "w1496_nuvigil"       "w3502_opportunity"   "w4087_market"       
 [19] "w3919_program"       "w3811_improve"       "w3717_develop"      
 [22] "w3635_achieve"       "w3643_competitive"   "w3656_offering"     
 [25] "w4072_expect"        "w3150_timely"        "w3386_low"          
 [28] "w3498_profitability" "w4073_continue"      "w3845_system"       
 [31] "w3558_focus"         "w3497_location"      "w3714_generation"   
 [34] "w2436_graphics"      "w3579_release"       "w3673_high"         
 [37] "w3836_marketing"     "w3905_line"          "w4074_plan"         
 [40] "w4066_additional"    "w3969_exist"         "w4062_quarter"      
 [43] "w4057_customer"      "w4078_cost"          "w2728_hdd"          
 [46] "w3229_invest"        "w4024_higher"        "w3600_design"       
 [49] "w3923_believe"       "w4120_financial"     "w4099_fiscal"       
 [52] "w3608_allow"         "w3786_limited"       "w3878_three"        
 [55] "w4084_record"        "w4091_future"        "w3903_february"     
 [58] "w4082_consolidate"   "w4106_statement"     "w4092_note"         
 [61] "w4126_increase"      "w4088_credit"        "w4117_due"          
 [64] "w4079_accounting"    "w4102_amount"        "w3689_network"      
 [67] "w3941_ability"       "w3994_decline"       "w3972_service"      
 [70] "w3240_successfully"  "w3835_dollar"        "w4107_income"       
 [73] "w4101_interest"      "w4104_estimate"      "w3914_demand"       
 [76] "w4103_facility"      "w4108_tax"           "w4053_fai

In [102]:
m_min=2
m_max = 4129
grid_length=20
grids = unique(round(exp(seq(log(m_min), log(m_max),
length=grid_length))))
grids


[1]    2    3    4    7   10   15   22   33   50   74  111  166  248  371  554
[16]  828 1237 1849 2763 4129

In [69]:
length(dstrain_y)
dim(dstrain)
dim(dstune)[2]
head(dstune)

[1] 619

[1]  619 4129

[1] 4129

,s.product,w1_geyser,w2_bluegrass,w3_irc,w4_pechiney,w5_pd,w6_irvine,w7_sonic,w8_cheshire,w9_entail,⋯,w4119_relate,w4120_financial,w4121_cash,w4122_december,w4123_revenue,w4124_expense,w4125_include,w4126_increase,w4127_result,w4128_numword
56768,pos,0,0,0,0,0,0,0,0,0,⋯,0.0000000,0,0.0000000,0.00000000,0.00000000,0.0000000,0,0.08200595,0.0000000,0.00000000
48875,neg,0,0,0,0,0,0,0,0,0,⋯,0.1242603,0,0.0000000,0.03068368,0.00000000,0.0292831,0,0.00000000,0.0000000,0.09353100
60905,neg,0,0,0,0,0,0,0,0,0,⋯,0.0000000,0,0.1506717,0.00000000,0.00000000,0.0000000,0,0.00000000,0.0000000,0.00000000
42006,neg,0,0,0,0,0,0,0,0,0,⋯,0.0000000,0,0.0000000,0.00000000,0.00000000,0.0000000,0,0.00000000,0.0000000,0.00000000
46638,neg,0,0,0,0,0,0,0,0,0,⋯,0.0000000,0,0.0000000,0.00000000,0.06973605,0.0000000,0,0.06621251,0.0000000,0.03381814
60057,neg,0,0,0,0,0,0,0,0,0,⋯,0.0000000,0,0.0000000,0.00000000,0.00000000,0.1380310,0,0.39839464,0.1213555,0.06782689
